# Create rotations chart data

# Setup

In [38]:
import pandas as pd
import numpy as np
import nba_api

In [39]:
from nba_api.stats.static.teams import get_teams, find_teams_by_nickname
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import PlayByPlayV2
from nba_api.stats.endpoints import PlayByPlay

In [40]:
spurs_info = find_teams_by_nickname('Spurs')
spurs_id = spurs_info[0]['id']

# find all Spurs games
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=spurs_id,
                                              season_nullable='2019-20')
team_box_scores_df = gamefinder.get_data_frames()[0]
team_box_scores_df.head(n=10)

spurs_games = list(team_box_scores_df.GAME_ID.unique()) # grab unique list of game IDs

test_game = spurs_games[0]

In [41]:
pbp = PlayByPlayV2(game_id=test_game).data_sets

In [42]:
pbp_df = pbp[0].get_data_frame()
pbp_df.head(n=10)

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG
0,0021901314,2,12,0,1,6:38 PM,12:00,None,None,None,...,None,None,0,0,None,NaN,None,None,None,0
1,0021901314,4,10,0,1,6:38 PM,12:00,Jump Ball Bradley vs. Poeltl: Tip to Ingles,None,None,...,Spurs,SAS,4,204060,Joe Ingles,1.610613e+09,Utah,Jazz,UTA,1
2,0021901314,7,1,47,1,6:38 PM,11:46,Clarkson 7' Turnaround Jump Shot (2 PTS) (Mitc...,None,None,...,Jazz,UTA,0,0,None,NaN,None,None,None,1
3,0021901314,9,2,78,1,6:38 PM,11:31,O'Neale BLOCK (1 BLK),None,MISS Samanic 4' Floating Jump Shot,...,None,None,4,1626220,Royce O'Neale,1.610613e+09,Utah,Jazz,UTA,1
4,0021901314,11,4,0,1,6:38 PM,11:28,Bradley REBOUND (Off:0 Def:1),None,None,...,None,None,0,0,None,NaN,None,None,None,1
5,0021901314,12,2,1,1,6:38 PM,11:23,MISS O'Neale 25' 3PT Jump Shot,None,None,...,None,None,0,0,None,NaN,None,None,None,1
6,0021901314,13,4,0,1,6:38 PM,11:17,None,None,Samanic REBOUND (Off:0 Def:1),...,None,None,0,0,None,NaN,None,None,None,1
7,0021901314,14,2,78,1,6:38 PM,11:12,None,None,MISS Walker IV 14' Floating Jump Shot,...,None,None,0,0,None,NaN,None,None,None,1
8,0021901314,15,4,0,1,6:38 PM,11:10,Bradley REBOUND (Off:0 Def:2),None,None,...,None,None,0,0,None,NaN,None,None,None,1
9,0021901314,16,2,1,1,6:39 PM,10:58,MISS Clarkson 25' 3PT Jump Shot,None,None,...,None,None,0,0,None,NaN,None,None,None,1


# Functions

In [43]:
def conv_time_to_sec(time):
    split_time = time.split(':')
    new_time = int(split_time[0]) * 60 + int(split_time[1])
    return new_time

In [81]:
def parse_player_in(pbp_text):
    split_text = pbp_text.split('FOR')
    
    return (split_text[0][4:].strip())

In [82]:
def parse_player_out(pbp_text):
    split_text = pbp_text.split('FOR')
    
    return (split_text[1].strip())

# Filtering & Exploration

In [45]:
pbp_subs = pbp_df[pbp_df['HOMEDESCRIPTION'].str.contains('SUB') == True].copy()
pbp_subs.head()

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG
45,0021901314,67,8,0,1,6:49 PM,6:16,SUB: Niang FOR Clarkson,None,None,...,Jazz,UTA,0,0,None,NaN,None,None,None,0
46,0021901314,68,8,0,1,6:49 PM,6:16,SUB: Oni FOR Ingles,None,None,...,Jazz,UTA,0,0,None,NaN,None,None,None,0
67,0021901314,101,8,0,1,6:57 PM,3:51,SUB: Ingles FOR Bradley,None,None,...,Jazz,UTA,0,0,None,NaN,None,None,None,0
68,0021901314,102,8,0,1,6:57 PM,3:51,SUB: Davis FOR O'Neale,None,None,...,Jazz,UTA,0,0,None,NaN,None,None,None,0
69,0021901314,103,8,0,1,6:57 PM,3:51,SUB: Clarkson FOR Mitchell,None,None,...,Jazz,UTA,0,0,None,NaN,None,None,None,0


In [46]:
pbp_subs.columns

Index(['GAME_ID', 'EVENTNUM', 'EVENTMSGTYPE', 'EVENTMSGACTIONTYPE', 'PERIOD',
       'WCTIMESTRING', 'PCTIMESTRING', 'HOMEDESCRIPTION', 'NEUTRALDESCRIPTION',
       'VISITORDESCRIPTION', 'SCORE', 'SCOREMARGIN', 'PERSON1TYPE',
       'PLAYER1_ID', 'PLAYER1_NAME', 'PLAYER1_TEAM_ID', 'PLAYER1_TEAM_CITY',
       'PLAYER1_TEAM_NICKNAME', 'PLAYER1_TEAM_ABBREVIATION', 'PERSON2TYPE',
       'PLAYER2_ID', 'PLAYER2_NAME', 'PLAYER2_TEAM_ID', 'PLAYER2_TEAM_CITY',
       'PLAYER2_TEAM_NICKNAME', 'PLAYER2_TEAM_ABBREVIATION', 'PERSON3TYPE',
       'PLAYER3_ID', 'PLAYER3_NAME', 'PLAYER3_TEAM_ID', 'PLAYER3_TEAM_CITY',
       'PLAYER3_TEAM_NICKNAME', 'PLAYER3_TEAM_ABBREVIATION',
       'VIDEO_AVAILABLE_FLAG'],
      dtype='object')

In [47]:
pbp_subs.dtypes

GAME_ID                       object
EVENTNUM                       int64
EVENTMSGTYPE                   int64
EVENTMSGACTIONTYPE             int64
PERIOD                         int64
WCTIMESTRING                  object
PCTIMESTRING                  object
HOMEDESCRIPTION               object
NEUTRALDESCRIPTION            object
VISITORDESCRIPTION            object
SCORE                         object
SCOREMARGIN                   object
PERSON1TYPE                    int64
PLAYER1_ID                     int64
PLAYER1_NAME                  object
PLAYER1_TEAM_ID              float64
PLAYER1_TEAM_CITY             object
PLAYER1_TEAM_NICKNAME         object
PLAYER1_TEAM_ABBREVIATION     object
PERSON2TYPE                    int64
PLAYER2_ID                     int64
PLAYER2_NAME                  object
PLAYER2_TEAM_ID              float64
PLAYER2_TEAM_CITY             object
PLAYER2_TEAM_NICKNAME         object
PLAYER2_TEAM_ABBREVIATION     object
PERSON3TYPE                    int64
P

In [48]:
pbp_subs['PCTIMESTRING'].dtypes

dtype('O')

In [49]:
conv_time_to_sec(pbp_subs['PCTIMESTRING'].iloc[0])

376

In [55]:
# convert time to seconds

pbp_subs['period_time'] = pbp_subs.PCTIMESTRING.apply(conv_time_to_sec)

# convert period time to total time
# period - 1 to account for extra 12 mins
pbp_subs['game_time'] = pbp_subs['period_time'] + (pbp_subs['PERIOD']-1) * 12 

In [56]:
pbp_subs.head()

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG,game_time,period_time
45,0021901314,67,8,0,1,6:49 PM,6:16,SUB: Niang FOR Clarkson,None,None,...,0,0,None,NaN,None,None,None,0,376,376
46,0021901314,68,8,0,1,6:49 PM,6:16,SUB: Oni FOR Ingles,None,None,...,0,0,None,NaN,None,None,None,0,376,376
67,0021901314,101,8,0,1,6:57 PM,3:51,SUB: Ingles FOR Bradley,None,None,...,0,0,None,NaN,None,None,None,0,231,231
68,0021901314,102,8,0,1,6:57 PM,3:51,SUB: Davis FOR O'Neale,None,None,...,0,0,None,NaN,None,None,None,0,231,231
69,0021901314,103,8,0,1,6:57 PM,3:51,SUB: Clarkson FOR Mitchell,None,None,...,0,0,None,NaN,None,None,None,0,231,231


In [85]:
filtered_pbp_subs = pbp_subs[['PERIOD', 'game_time', 'HOMEDESCRIPTION']].copy()

In [86]:
# create subs chart
filtered_pbp_subs['player_in'] = filtered_pbp_subs['HOMEDESCRIPTION'].apply(parse_player_in)
filtered_pbp_subs['player_out'] = filtered_pbp_subs['HOMEDESCRIPTION'].apply(parse_player_out)

In [91]:
subs_out_df = filtered_pbp_subs[['PERIOD', 'game_time', 'player_in', 'player_out']].copy()
subs_out_df

,PERIOD,game_time,player_in,player_out
45,1,376,Niang,Clarkson
46,1,376,Oni,Ingles
67,1,231,Ingles,Bradley
68,1,231,Davis,O'Neale
69,1,231,Clarkson,Mitchell
70,1,231,Morgan,Oni
140,2,567,Mitchell,Ingles
141,2,567,O'Neale,Morgan
142,2,567,Oni,Niang
155,2,516,Brantley,Clarkson


In [90]:
subs_out_df.to_json('data/subs_data.json', orient='records')

In [93]:
subs_out_df.to_json('subs_data.json', orient='index')